In [1]:
from scipy.io import loadmat
# Zero Padded Data
# data = loadmat("/content/drive/MyDrive/EEG_full_4D_1Hz.mat")
# Read data in Kaggle version 
data = loadmat("/kaggle/input/eeg-dataset/EEG_full_4D_1Hz.mat")
print("done")

done


In [2]:
epoch_num = data['epoch_num']
epoch_num

array([[46, 58, 27, 21, 36, 47, 48, 20, 41, 73, 36, 20, 48, 16, 74, 46,
        44, 45, 33, 30, 57, 19, 48, 50, 19, 19, 37, 67, 29, 30, 59, 57,
        29, 57, 22, 66, 38, 29, 48, 31, 35, 62, 44, 73, 26, 51, 23, 28,
        65, 13, 30, 43, 34, 50, 46, 61, 34, 50, 42, 29, 16, 34, 48, 59,
        29, 16, 30, 26, 47, 19, 45, 12, 52, 25, 35, 41, 29, 37, 35, 42,
        39, 24, 18,  3, 24,  4, 27, 49]], dtype=uint8)

In [3]:
import numpy as np

# Inspect the EEG_Class structure
eeg_data = data['EEG']
eeg_data.shape

(65, 2500, 19, 74)

# Divide Data

In [4]:
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential, Model, clone_model
from keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Input, ConvLSTM2D,MaxPooling3D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback, LearningRateScheduler
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, InceptionV3, DenseNet121, EfficientNetB2
from tensorflow.keras.layers import LSTM, GRU, TimeDistributed, Input
from tensorflow.keras.models import load_model
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import shutil
import scipy

In [14]:
lobes = {
    'Frontal': [1, 2, 3, 4, 11, 12, 17],
    'Temporal': [13, 14, 15, 16],
    'Parietal': [7, 8, 19],
    'Central': [5, 6, 18],
    'Occipital': [9, 10]
}


######################### KAGGLE VERSION ###############################

import os
# Iterate over each subject's data
for subject_idx in range(eeg_data.shape[0]):
    subject_data = eeg_data[subject_idx, :, :,:]  # EEG data for the current subject

    # Create directory to save time-series
    save_dir = os.path.join('/kaggle/working/','Alzheimer','Time_series')
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Iterate over each channel
    # channels_of_interest = [6,7, 12,13]

    #for channel_idx in channels_of_interest:
    for channel_idx in range(subject_data.shape[1]):
        channel_data = subject_data[:,channel_idx, :]  # EEG data for the current channel

        # Iterate over each 10-second segment
        # for seg_idx in range(epoch_num[0,subject_idx]):
        for seg_idx in range(10):

            segment_data = channel_data[:,seg_idx]      # EEG data for the current Segment

            # Save all time-series
            save_path = os.path.join(save_dir, 'All lobes',f'subject{subject_idx + 1}')
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            save_path2 = os.path.join(save_path, f'subject{subject_idx + 1}_channel{channel_idx + 1}_segment{seg_idx + 1}')
            np.save(save_path2,segment_data)



            # Save spectrogram image for each lobe
            for lobe_name, lobe_channels in lobes.items():
                if channel_idx + 1 in lobe_channels:
                    save_dir2 = os.path.join('/kaggle/working/', 'Alzheimer', 'Time_series', lobe_name,f'subject{subject_idx + 1}')
                    if not os.path.exists(save_dir2):
                        os.makedirs(save_dir2)
                    save_path = os.path.join(save_dir2, f'subject{subject_idx + 1}_channel{channel_idx + 1}_segment{seg_idx + 1}')

                    np.save(save_path, segment_data)


print("saved successfully.")

saved successfully.


# Data Augmentation 

In [11]:
# import numpy as np
# from scipy.interpolate import interp1d

# def standardize_length(signal, target_length=2500):
#     current_length = len(signal)
#     if current_length < target_length:
#         # Pad the signal if it is shorter than the target length
#         padding = np.zeros(target_length - current_length)
#         signal = np.concatenate((signal, padding))
#     elif current_length > target_length:
#         # Trim the signal if it is longer than the target length
#         signal = signal[:target_length]
#     return signal


# def load_eeg_data(file_path):
#     # Assuming the EEG data is stored in a NumPy binary file (.npy). Adjust this as necessary.
#     return np.load(file_path)


# def time_reverse(signal, probability=0.5):
#     if np.random.rand() < probability:
#         return signal[::-1]
#     return signal


# def smooth_time_mask(signal, mask_length, probability=0.5):
#     if np.random.rand() < probability:
#         if mask_length < len(signal):
#             start = np.random.randint(0, len(signal) - mask_length)
#             signal[start:start + mask_length] = 0
#         else:
#             # Handle case where mask length is inappropriate
#             print("Mask length is too long for the signal size.")
#     return signal



# def gaussian_noise(signal, sigma=0.05, probability=0.5):
#     if np.random.rand() < probability:
#         noise = np.random.normal(0, sigma, size=len(signal))
#         return signal + noise
#     return signal




# def channel_dropout(signal, dropout_prob=0.1):
#     if signal.ndim == 2:  # Multi-channel data
#         num_channels = signal.shape[1]
#         if np.random.rand() < dropout_prob and num_channels > 1:
#             channels_to_drop = np.random.choice(num_channels, size=np.random.randint(1, num_channels), replace=False)
#             signal[:, channels_to_drop] = 0
#     return signal


# def time_warp(signal, time_warping_prob=0.5):
#     if np.random.rand() < time_warping_prob:
#         length = len(signal)
#         original_indices = np.arange(length)
#         warp_points = np.sort(np.random.choice(original_indices[1:-1], size=2, replace=False))
#         warp_factors = np.random.uniform(0.5, 1.5, size=2)
        
#         new_indices = np.concatenate([
#             np.linspace(0, warp_points[0], num=int(warp_factors[0] * warp_points[0]), endpoint=False),
#             np.linspace(warp_points[0], warp_points[1], num=int(warp_factors[1] * (warp_points[1] - warp_points[0])), endpoint=False),
#             np.linspace(warp_points[1], length, num=int((length - warp_points[1]) * warp_factors[1]))
#         ])
        
#         new_indices = np.clip(new_indices, 0, length-1)
#         new_indices = np.round(new_indices).astype(int)
        
#         if signal.ndim == 1:  # Single-channel
#             interpolation_function = interp1d(original_indices, signal, kind='linear', fill_value='extrapolate')
#             signal = interpolation_function(original_indices)
#         else:  # Multi-channel
#             for i in range(signal.shape[1]):
#                 interpolation_function = interp1d(original_indices, signal[:, i], kind='linear', fill_value='extrapolate')
#                 signal[:, i] = interpolation_function(original_indices)
        
#     return signal





# def amplitude_scaling(signal, scaling_prob=0.5, min_scale=0.8, max_scale=1.2):
#     if np.random.rand() < scaling_prob:
#         scale_factor = np.random.uniform(min_scale, max_scale)
#         signal = signal * scale_factor
#     return signal


# # def augment_eeg_signal(signal, mask_length=50, noise_sigma=0.05, time_rev_prob=0.5, mask_prob=0.5, noise_prob=0.5):
# #     signal = load_eeg_data(file_path)  # Load data
# #     # Apply TimeReverse
# #     signal = time_reverse(signal, probability=time_rev_prob)

# #     # Apply SmoothTimeMask
# #     signal = smooth_time_mask(signal, mask_length=mask_length, probability=mask_prob)

# #     # Apply GaussianNoise
# #     signal = gaussian_noise(signal, sigma=noise_sigma, probability=noise_prob)

# #     return signal


# def augment_eeg_signal(signal_path, mask_length=50, noise_sigma=0.05, time_rev_prob=0.5, mask_prob=0.5, noise_prob=0.5, target_length=2500):
#     signal = load_eeg_data(signal_path)  # Load data
#     #print("Original:", signal.shape)
#     signal = standardize_length(signal, target_length=target_length)
#     #print("After Length Standardization:", signal.shape)
#     signal = time_reverse(signal, probability=time_rev_prob)
#     #print("After Time Reverse:", signal.shape)

#     signal = smooth_time_mask(signal, mask_length=mask_length, probability=mask_prob)
#     #print("After Smooth Time Mask:", signal.shape)

#     signal = gaussian_noise(signal, sigma=noise_sigma, probability=noise_prob)
#     #print("After Gaussian Noise:", signal.shape)

#     signal = amplitude_scaling(signal)
#     #print("After Amplitude Scaling:", signal.shape)

#     signal = channel_dropout(signal)
#     #print("After Channel Dropout:", signal.shape)

#     signal = time_warp(signal, time_warping_prob=0.5)
#     #print("After Time Warp:", signal.shape)

#     # Standardize the length of the signal to ensure consistency
    

#     return signal






# import os 
# data_dir = "/kaggle/working/Alzheimer/Time_series/All lobes"
# folders = os.listdir(data_dir)
# for folder in folders:
#   address = os.path.join(data_dir, folder)
#   particular_subject_folder = os.listdir(address)
#   for file in particular_subject_folder:
#     if file.endswith(".npy"):
#         file_path = os.path.join(address, file)
#         sig = augment_eeg_signal(file_path)
#         save_path = os.path.join(address, "Augmented_" + file)
#         np.save(save_path, sig)
# print("Data Augmentation completed Success fully")

Data Augmentation completed Success fully


# Data Entering

In [15]:
# Run this for Colab
# data_dir = "/content/Alzheimer/Time_series/Temporal"
# Run this for Kaggle
data_dir = "/kaggle/working/Alzheimer/Time_series/Parietal"

def extract_numeric_part(folder_name):
    parts = folder_name.split('_')
    for part in parts:
        if part.startswith('subject'):
            return int(part.replace('subject', ''))
    return -1  # Return -1 if no numeric part is found

folders = sorted([f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))], key=extract_numeric_part)

subject_folders, labels = [], []

for folder_name in folders:
    subject_num = extract_numeric_part(folder_name)
    subject_folder = os.path.join(data_dir, folder_name)
    subject_folders.append(subject_folder)

    if subject_num <= 36:
        labels.append(0)  # AD group
    elif 37 <= subject_num <= 65:
        labels.append(1)  # HC group

subject_folders = np.array(subject_folders)
labels = np.array(labels)

print("Subject Folders:", subject_folders)
print("Labels:", labels)

Subject Folders: ['/kaggle/working/Alzheimer/Time_series/Parietal/subject1'
 '/kaggle/working/Alzheimer/Time_series/Parietal/subject2'
 '/kaggle/working/Alzheimer/Time_series/Parietal/subject3'
 '/kaggle/working/Alzheimer/Time_series/Parietal/subject4'
 '/kaggle/working/Alzheimer/Time_series/Parietal/subject5'
 '/kaggle/working/Alzheimer/Time_series/Parietal/subject6'
 '/kaggle/working/Alzheimer/Time_series/Parietal/subject7'
 '/kaggle/working/Alzheimer/Time_series/Parietal/subject8'
 '/kaggle/working/Alzheimer/Time_series/Parietal/subject9'
 '/kaggle/working/Alzheimer/Time_series/Parietal/subject10'
 '/kaggle/working/Alzheimer/Time_series/Parietal/subject11'
 '/kaggle/working/Alzheimer/Time_series/Parietal/subject12'
 '/kaggle/working/Alzheimer/Time_series/Parietal/subject13'
 '/kaggle/working/Alzheimer/Time_series/Parietal/subject14'
 '/kaggle/working/Alzheimer/Time_series/Parietal/subject15'
 '/kaggle/working/Alzheimer/Time_series/Parietal/subject16'
 '/kaggle/working/Alzheimer/Time

In [16]:
def load_series_and_labels(subjects, labels):
    series, labels_t = [], []
    for subject_folder, label in zip(subjects, labels):
        for filename in os.listdir(subject_folder):
            series_path = os.path.join(subject_folder, filename.decode())
            try:
                ser = np.load(series_path)


                series.append(ser)
                labels_t.append(label)
            except Exception as e:
                print(f"Error loading time-series: {series_path}, {e}")

    return np.array(series), np.array(labels_t)

In [17]:
def load_and_preprocess_data(subject_folders, labels):
    # Create dictionaries to categorize subjects into train, val, and test groups
    train_subjects_dict = {}
    val_subjects_dict = {}
    test_subjects_dict = {}

    for subject_folder, label in zip(subject_folders, labels):
        subject_num = extract_numeric_part(os.path.basename(subject_folder))
        if 33 <= subject_num <= 41 :
            val_subjects_dict[subject_folder] = label
        elif 27 <= subject_num  <= 32 or 42 <= subject_num <= 48:
            test_subjects_dict[subject_folder] = label
        else:
            train_subjects_dict[subject_folder] = label

    # Extract subjects and labels for each group
    train_subjects = list(train_subjects_dict.keys())
    train_labels = [train_subjects_dict[subj] for subj in train_subjects]
    val_subjects = list(val_subjects_dict.keys())
    val_labels = [val_subjects_dict[subj] for subj in val_subjects]
    test_subjects = list(test_subjects_dict.keys())
    test_labels = [test_subjects_dict[subj] for subj in test_subjects]

    # Load images for each group
    train_images, train_labels_t = load_series_and_labels(train_subjects, train_labels)
    val_images, val_labels_t = load_series_and_labels(val_subjects, val_labels)
    test_images, test_labels_t = load_series_and_labels(test_subjects, test_labels)
    test_image_counts_per_subject = [len(os.listdir(folder)) for folder in test_subjects]

    return train_images, train_labels_t, val_images, val_labels_t, test_images, test_labels_t, test_subjects, test_image_counts_per_subject


# ResNet

In [18]:
train_series, train_labels_t, val_series, val_labels_t, test_series, test_labels_t, test_subjects, test_image_counts_per_subject = load_and_preprocess_data(subject_folders, labels)

In [19]:
from tensorflow.keras.initializers import HeNormal
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, ReLU, Add, GlobalAveragePooling1D, Dense, Dropout, Multiply, Reshape

def se_block(input_tensor, reduction_ratio=8):
    """Squeeze-and-Excitation block to add attention."""
    filters = input_tensor.shape[-1]
    se_shape = (1, filters)
    
    # Squeeze operation
    se = GlobalAveragePooling1D()(input_tensor)
    se = Reshape(se_shape)(se)
    
    # Excitation operation
    se = Dense(filters // reduction_ratio, activation='relu', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', use_bias=False)(se)
    
    # Scaling the input_tensor with the learned attention
    x = Multiply()([input_tensor, se])
    return x

def residual_block(x, filters, kernel_size=3, stride=1, dilation_rate=1):
    """A residual block with SE block and dilated convolution for attention."""
    shortcut = x
    
    # First convolution with dilation
    x = Conv1D(filters, kernel_size=kernel_size, strides=stride, padding='same', dilation_rate=dilation_rate, kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = ReLU()(x)
    
    # Second convolution with dilation
    x = Conv1D(filters, kernel_size=kernel_size, strides=1, padding='same', dilation_rate=dilation_rate, kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    # new layer
    #x = LSTM(64, return_sequences=True)(x)
    #x = LSTM(64, return_sequences=True)(x)
    #x = LSTM(64, return_sequences=True)(x)
    # new layer
    
    # Adding the shortcut to the output of the convolutions
    if x.shape[-1] != shortcut.shape[-1] or stride != 1:
        shortcut = Conv1D(filters, kernel_size=1, strides=stride, padding='same', kernel_initializer='he_normal')(shortcut)
        shortcut = BatchNormalization()(shortcut)
        
    x = Add()([x, shortcut])
    x = ReLU()(x)
    
    # Apply SE block (attention mechanism)
    x = se_block(x)
    
    return x

def build_resnet(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    
    # Initial convolution
    x = Conv1D(64, kernel_size=3, strides=1, padding='same', kernel_initializer=HeNormal())(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.7)(x)
    x = ReLU()(x)
    
    # Adding residual blocks with attention
    x = residual_block(x, 64, stride=1, dilation_rate=1)
    x = residual_block(x, 128, stride=1, dilation_rate=2)
    x = residual_block(x, 256, stride=1, dilation_rate=4)
    x = residual_block(x, 512, stride=1, dilation_rate=8)
    x = residual_block(x, 1024, stride=1, dilation_rate=16)
    
    # Classifier
    x = GlobalAveragePooling1D()(x)
    x = Dense(1024, activation='relu', kernel_initializer=HeNormal())(x)
    x = Dropout(0.1)(x)
    x = Dense(512, activation='relu', kernel_initializer=HeNormal())(x)
    x = Dropout(0.1)(x)

    outputs = Dense(num_classes, activation='sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    # Compile the model with Adam optimizer and learning rate scheduling
    # optimizer = Adam(learning_rate=0.001)
    # model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model


# Build the model
input_shape = (2500, 1)  # Change according to your EEG data shape
num_classes = 1  # Change to the number of classes you have
model = build_resnet(input_shape, num_classes)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.007)

model.compile(optimizer=optimizer, 
                      loss='binary_crossentropy', metrics=['accuracy'])
# Summary of the model
model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 2500, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_15 (Conv1D)  │ (None, 2500, 64)  │        256 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 2500, 64)  │        256 │ conv1d_15[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_13          │ (None, 2500, 64)  │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_11 (ReLU)     │ (None, 2500, 64)  │          0 │ dropout_13[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_16 (Conv1D)  │ (None, 2500, 64)  │     12,352 │ re_lu_11[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 2500, 64)  │        256 │ conv1d_16[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_14          │ (None, 2500, 64)  │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_12 (ReLU)     │ (None, 2500, 64)  │          0 │ dropout_14[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_17 (Conv1D)  │ (None, 2500, 64)  │     12,352 │ re_lu_12[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 2500, 64)  │        256 │ conv1d_17[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_15          │ (None, 2500, 64)  │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_5 (Add)         │ (None, 2500, 64)  │          0 │ dropout_15[0][0], │
│                     │                   │            │ re_lu_11[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_13 (ReLU)     │ (None, 2500, 64)  │          0 │ add_5[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ re_lu_13[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_5 (Reshape) │ (None, 1, 64)     │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 1, 8)      │        512 │ reshape_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 1, 64)     │        512 │ dense_13[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_5          │ (None, 2500, 64)  │          0 │ re_lu_13[0][0],   │
│ (Multiply)          │                   │            │ dense_14[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_18 (Conv1D)  │ (None, 2500, 128) │     24,704 │ multiply_5[0][0]

 Total params: 8,941,825 (34.11 MB)

 Trainable params: 8,929,921 (34.06 MB)

 Non-trainable params: 11,904 (46.50 KB)

# W/O iterations

In [ ]:
import numpy as np
import os
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint 
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

fold_no = 1
scores = []
keras = tf.keras
hist = []
model = model
# Create and train the model



# Checkpoint to save the best model in each fold
filepath = f"/kaggle/working/best_model_folds_{fold_no}.keras"
if os.path.exists(filepath):
    print("File exists")
else:
    print("File does not exist")


checkpoint = ModelCheckpoint(filepath,
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             mode='max',
                            initial_value_threshold = None)


lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.1,  # Factor by which the learning rate will be reduced
    patience=6,  # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-6  # Lower bound on the learning rate

)

early_stopping = EarlyStopping(monitor='val_accuracy',
                               patience=6, 
                               verbose=1,
                               mode='max', 
                               baseline=None,
                               restore_best_weights=True, 
                               start_from_epoch=25)

hist_temp = model.fit(train_series, train_labels_t, validation_data=(val_series, val_labels_t),
                     epochs=70,
                     batch_size=8, 
                     callbacks=[checkpoint, lr_scheduler, early_stopping],
                     verbose=1)
#model.save(filepath)  # Saves as a SavedModel directory

hist.append(hist_temp)
# Load the best model and evaluate it




model.load_weights(filepath)
score = model.evaluate(test_series, test_labels_t, verbose=1)
scores.append(score[1])  # Assuming 1 is accuracy
print(f"\nScore for fold {fold_no}: {model.metrics_names[1]} of {score[1]}; {model.metrics_names[0]} of {score[0]}")
# calculate metrics = precision, recall, ...

test_predictions = model.predict(test_series)
test_predictions = (test_predictions > 0.5).astype(int)

accuracy = accuracy_score(test_labels_t, test_predictions)
precision = precision_score(test_labels_t, test_predictions)
recall = recall_score(test_labels_t, test_predictions)
f1 = f1_score(test_labels_t, test_predictions)

# accuracy_list.append(accuracy)
# precision_list.append(precision)
# recall_list.append(recall)
# f1_list.append(f1)

print("\nPredictions for Each Subject's Images:")
start_idx = 0
for i, (subject_folder, num_images) in enumerate(zip(test_subjects, test_image_counts_per_subject)):
        end_idx = start_idx + num_images
        subject_images = test_series[start_idx:end_idx]
        subject_predictions = (model.predict(subject_images) > 0.5).astype(int)
        ad_count = sum(subject_predictions == 0)
        hc_count = sum(subject_predictions == 1)
        print(f"Subject {i + 1} ({subject_folder}):")
        print(f"AD count: {ad_count}, HC count: {hc_count}")
        start_idx = end_idx


# plot the curves for validation and loss 


# Extract history data
history = hist_temp.history
epochs = range(1, len(history['loss']) + 1)  # Number of epochs

# Plot training and validation accuracy
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, history['accuracy'], label='Training Accuracy', marker='o')
plt.plot(epochs, history['val_accuracy'], label='Validation Accuracy', marker='o')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(epochs, history['loss'], label='Training Loss', marker='o')
plt.plot(epochs, history['val_loss'], label='Validation Loss', marker='o')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()



#fold_no += 1
time.sleep(30)

# Calculate and print the average score
# average_score = np.mean(scores)
# average_accuracy = np.mean(accuracy_list)
# average_precision = np.mean(precision_list)
# average_recall = np.mean(recall_list)
# average_f1 = np.mean(f1_list)
# print(f"\nAverage Score: {average_score}")
# calculate metrics = precision, recall, ...


# print("\nAverage Metrics:")
# print(f"Average Precision: {np.mean(average_accuracy):.4f}")
# print(f"Average Precision: {np.mean(average_precision):.4f}")
# print(f"Average Recall: {np.mean(average_recall):.4f}")
# print(f"Average F1 Score: {np.mean(average_f1):.4f}")

# with iterations

In [20]:
import numpy as np
import os
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint 
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import load_model
 
iterations = 5
scores = []
keras = tf.keras
hist = []
model = model
# Create and train the model
acc_epoch_list = []
precision_epoch_list = []
f1_epoch_list = [] 
recall_epoch_list = []
acc_sbj_list = []
precision_sbj_list = []
f1_sbj_list = []
recall_sbj_list = []

for iteration in range(iterations):

    # Checkpoint to save the best model in each fold
    filepath = f"/kaggle/working/best_model_folds_{iteration}.keras"
    if os.path.exists(filepath):
        print("File exists")
    else:
        print("File does not exist")


    checkpoint = ModelCheckpoint(filepath,
                                 monitor='val_accuracy',
                                 verbose=1,
                                 save_best_only=True,
                                 mode='max',
                                initial_value_threshold = None)


    lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.1,  # Factor by which the learning rate will be reduced
        patience=6,  # Number of epochs with no improvement after which learning rate will be reduced
        min_lr=1e-6  # Lower bound on the learning rate

    )

    early_stopping = EarlyStopping(monitor='val_accuracy',
                                   patience=6, 
                                   verbose=1,
                                   mode='max', 
                                   baseline=None,
                                   restore_best_weights=True, 
                                   start_from_epoch=20)

    hist_temp = model.fit(train_series, train_labels_t, validation_data=(val_series, val_labels_t),
                         epochs=60,
                         batch_size=8, 
                         callbacks=[checkpoint, lr_scheduler, early_stopping],
                         verbose=1)
    #model.save(filepath)  # Saves as a SavedModel directory

    hist.append(hist_temp)
    # Load the best model and evaluate it




    model.load_weights(filepath)
    score = model.evaluate(test_series, test_labels_t, verbose=1)
    scores.append(score[1])  # Assuming 1 is accuracy
    #print(f"\nScore for fold {fold_no}: {model.metrics_names[1]} of {score[1]}; {model.metrics_names[0]} of {score[0]}")
    # calculate metrics = precision, recall, ...

    test_predictions = model.predict(test_series)
    test_predictions = (test_predictions > 0.5).astype(int)
    
    accuracy = accuracy_score(test_labels_t, test_predictions)
    precision = precision_score(test_labels_t, test_predictions)
    recall = recall_score(test_labels_t, test_predictions)
    f1 = f1_score(test_labels_t, test_predictions)
    print(f"Accuracy for epoch: {accuracy:.4f}, Precision for epoch: {precision:.4f}, Recall for epoch: {recall:.4f}, F1 for epoch: {f1:.4f}")

    acc_epoch_list.append(accuracy)
    precision_epoch_list.append(precision)
    recall_epoch_list.append(recall)
    f1_epoch_list.append(f1)
    

    
    print("\nPredictions for Each Subject's Images:")
    start_idx = 0
    predicted_subjects_list = []
    groundtruth_subjects_list = []
    for i, (subject_folder, num_images) in enumerate(zip(test_subjects, test_image_counts_per_subject)):
            end_idx = start_idx + num_images
            subject_images = test_series[start_idx:end_idx]
            subject_predictions = (model.predict(subject_images) > 0.5).astype(int)
            ad_count = sum(subject_predictions == 0)
            hc_count = sum(subject_predictions == 1)
            print(f"Subject {i + 1} ({subject_folder}):")
            print(f"AD count: {ad_count}, HC count: {hc_count}")
            start_idx = end_idx
            num = subject_folder[int(subject_folder.find("subject"))+7:]
            if int(num) < 37: 
                groundtruth_subjects_list.append(0) # AD groundtruth
            else: 
                groundtruth_subjects_list.append(1) # HC groundtruth

            start_idx = end_idx

            if ad_count > hc_count: 
                predicted_subjects_list.append(0) # AD predicted
            else:
                predicted_subjects_list.append(1) # HC predicted
            


    accuracy_sbj = accuracy_score(groundtruth_subjects_list, predicted_subjects_list)
    precision_sbj = precision_score(groundtruth_subjects_list, predicted_subjects_list)
    recall_sbj = recall_score(groundtruth_subjects_list, predicted_subjects_list)
    f1_sbj = f1_score(groundtruth_subjects_list, predicted_subjects_list)

    print(f"Accuracy for subject: {accuracy_sbj:.4f}, Precision for subject: {precision_sbj:.4f}, Recall for subject: {recall_sbj:.4f}, F1 for subject: {f1_sbj:.4f}")

    # plot the curves for validation and loss 
    # Extract history data
    history = hist_temp.history
    epochs = range(1, len(history['loss']) + 1)  # Number of epochs
    
    acc_sbj_list.append(accuracy_sbj)
    precision_sbj_list.append(precision_sbj)
    recall_sbj_list.append(recall_sbj)
    f1_sbj_list.append(f1_sbj)
    


    # Plot training and validation accuracy
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['accuracy'], label='Training Accuracy', marker='o')
    plt.plot(epochs, history['val_accuracy'], label='Validation Accuracy', marker='o')
    plt.title(f'Training and Validation Accuracy for iteration:{iteration}')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot training and validation loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['loss'], label='Training Loss', marker='o')
    plt.plot(epochs, history['val_loss'], label='Validation Loss', marker='o')
    plt.title(f'Training and Validation Loss for iteration:{iteration}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()



    time.sleep(90)
print(f'\nmean for epoch, accuracy {np.mean(acc_epoch_list)} and std is {np.std(acc_epoch_list)}')
print(f'\nmean for epoch, precision {np.mean(precision_epoch_list)} and std is {np.std(precision_epoch_list)}')
print(f'\nmean for epoch, recall {np.mean(recall_epoch_list)} and std is {np.std(recall_epoch_list)}')
print(f'\nmean for epoch, f1-score {np.mean(f1_epoch_list)} and std is {np.std(f1_epoch_list)}')
    
print(f'\nmean for subject, accuracy {np.mean(acc_sbj_list)} and std is {np.std(acc_sbj_list)}')
print(f'\nmean for subject, precision {np.mean(precision_sbj_list)} and std is {np.std(precision_sbj_list)}')
print(f'\nmean for subject, recall {np.mean(recall_sbj_list)} and std is {np.std(recall_sbj_list)}')
print(f'\nmean for subject, f1-score {np.mean(f1_sbj_list)} and std is {np.std(f1_sbj_list)}')
# Calculate and print the average score
# average_score = np.mean(scores)
# average_accuracy = np.mean(accuracy_list)
# average_precision = np.mean(precision_list)
# average_recall = np.mean(recall_list)
# average_f1 = np.mean(f1_list)
# print(f"\nAverage Score: {average_score}")
# calculate metrics = precision, recall, ...


# print("\nAverage Metrics:")
# print(f"Average Precision: {np.mean(average_accuracy):.4f}")
# print(f"Average Precision: {np.mean(average_precision):.4f}")
# print(f"Average Recall: {np.mean(average_recall):.4f}")
# print(f"Average F1 Score: {np.mean(average_f1):.4f}")

File exists
Epoch 1/60
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step - accuracy: 0.6094 - loss: 1.4848
Epoch 1: val_accuracy improved from -inf to 0.72593, saving model to /kaggle/working/best_model_folds_0.keras
162/162 ━━━━━━━━━━━━━━━━━━━━ 100s 391ms/step - accuracy: 0.6098 - loss: 1.4806 - val_accuracy: 0.7259 - val_loss: 0.7464 - learning_rate: 0.0070
Epoch 2/60
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.7256 - loss: 0.5414
Epoch 2: val_accuracy did not improve from 0.72593
162/162 ━━━━━━━━━━━━━━━━━━━━ 27s 169ms/step - accuracy: 0.7255 - loss: 0.5414 - val_accuracy: 0.5963 - val_loss: 1.6966 - learning_rate: 0.0070
Epoch 3/60
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.7424 - loss: 0.5177
Epoch 3: val_accuracy did not improve from 0.72593
162/162 ━━━━━━━━━━━━━━━━━━━━ 27s 169ms/step - accuracy: 0.7425 - loss: 0.5175 - val_accuracy: 0.5667 - val_loss: 1.5843 - learning_rate: 0.0070
Epoch 4/60
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.7538 - loss

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




mean for epoch, accuracy 0.7948717948717949 and std is 0.03307592922378891

mean for epoch, precision 0.8828243095140469 and std is 0.0326525455436973

mean for epoch, recall 0.719047619047619 and std is 0.10052018896730486

mean for epoch, f1-score 0.7870498556965952 and std is 0.047805105050596236

mean for subject, accuracy 0.8153846153846154 and std is 0.07844645405527362

mean for subject, precision 0.9016666666666667 and std is 0.08373237791386978

mean for subject, recall 0.7428571428571429 and std is 0.13997084244475305

mean for subject, f1-score 0.8071794871794872 and std is 0.08771179870360186


In [35]:
np.mean(acc_sbj_list)


0.9076923076923078